In [10]:
!pip install rdkit
!pip install mordred
!pip install git+https://github.com/samoturk/mol2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/samoturk/mol2vec to /tmp/pip-req-build-ib3p1pk1
  Running command git clone --filter=blob:none --quiet https://github.com/samoturk/mol2vec /tmp/pip-req-build-ib3p1pk1
  Resolved https://github.com/samoturk/mol2vec to commit 850d944d5f48a58e26ed0264332b5741f72555aa
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.5 MB/s eta 0:00:00
  Created wheel for mol2vec: filename=mol2vec-0.1-py3-none-any.whl size=14038 sha256=4b597b023928a3dca742d3dfc0d94ce853756402c5964fbed8c03c7e3a3beb9e
  Stored in directory: /tmp/pip-ephem-wheel-cache-c7k5og8r/wheels/5e/f4/f7/e6aa5c5af53710f28c7346042d7f567a5b45d97b694f825f17
Successfully 

In [2]:
import numpy as np
import pandas as pd
import rdkit
from rdkit import Chem, rdBase, DataStructs
from rdkit.Chem import rdmolfiles, rdmolops, AllChem, Draw, rdDepictor, Descriptors
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.Fingerprints import FingerprintMols

from IPython.display import SVG
from IPython.display import HTML
from IPython.display import display

from collections import defaultdict
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('./drive/MyDrive/AI/Material Informatics (MI)/Ring Repeating Units/Table of Supporting Information(SMILES)- Cal band gap_(RU1, RU2, RRU).csv')

In [5]:
df.head()

,NO.,Repeating units_RU1,Repeating units_RU2,SMILES_RU1,SMILES_RU2,SMILES_RRU,DFT band gap/eV
0,1,CH2-CO-NH-CS,CO-NH-CS-CH2,*-CC(=O)NC(-*)=S,*-CC(=S)NC(-*)=O,O=C1CC(=S)N1,2.691
1,2,CH2-CS-C6H4-O,CS-C6H4-O-CH2,*-CC(=S)C1=CC=C(O-*)C=C1,*-COC1=CC=C(C=C1)C(-*)=S,S=C1COC2=CC=C1C=C2,2.041
2,3,C6H4-C6H4-C4H2S-CS,C6H4-C4H2S-CS-C6H4,*-C(=S)C1=CC=C(S1)C1=CC=C(C=C1)C1=CC=C(-*)C=C1,*-C1=CC=C(C=C1)C(=S)C1=CC=C(S1)C1=CC=C(-*)C=C1,S=C1C2=CC=C(S2)C2=CC=C(C=C2)C2=CC=C1C=C2,1.807
3,4,CO-O-C4H2S-CS,O-C4H2S-CS-CO,*-C(=O)OC1=CC=C(S1)C(-*)=S,*-OC1=CC=C(S1)C(=S)C(-*)=O,O=C1OC2=CC=C(S2)C1=S,1.918
4,5,C6H4-CS-C4H2S-O,CS-C4H2S-O-C6H4,*-OC1=CC=C(S1)C(=S)C1=CC=C(-*)C=C1,*-C(=S)C1=CC=C(OC2=CC=C(-*)C=C2)S1,S=C1C2=CC=C(OC3=CC=C1C=C3)S2,1.930


In [6]:
df["mol_ru1"] = df["SMILES_RU1"].apply(lambda x:Chem.MolFromSmiles(x))
df["mol_ru2"] = df["SMILES_RU2"].apply(lambda x:Chem.MolFromSmiles(x))
df["mol_rru"] = df["SMILES_RRU"].apply(lambda x:Chem.MolFromSmiles(x))

In [7]:
df.head()

,NO.,Repeating units_RU1,Repeating units_RU2,SMILES_RU1,SMILES_RU2,SMILES_RRU,DFT band gap/eV,mol_ru1,mol_ru2,mol_rru
0,1,CH2-CO-NH-CS,CO-NH-CS-CH2,*-CC(=O)NC(-*)=S,*-CC(=S)NC(-*)=O,O=C1CC(=S)N1,2.691,<rdkit.Chem.rdchem.Mol object at 0x7fed97267a50>,<rdkit.Chem.rdchem.Mol object at 0x7fed972167b0>,<rdkit.Chem.rdchem.Mol object at 0x7fed972225f0>
1,2,CH2-CS-C6H4-O,CS-C6H4-O-CH2,*-CC(=S)C1=CC=C(O-*)C=C1,*-COC1=CC=C(C=C1)C(-*)=S,S=C1COC2=CC=C1C=C2,2.041,<rdkit.Chem.rdchem.Mol object at 0x7fed97267970>,<rdkit.Chem.rdchem.Mol object at 0x7fed97216890>,<rdkit.Chem.rdchem.Mol object at 0x7fed972226d0>
2,3,C6H4-C6H4-C4H2S-CS,C6H4-C4H2S-CS-C6H4,*-C(=S)C1=CC=C(S1)C1=CC=C(C=C1)C1=CC=C(-*)C=C1,*-C1=CC=C(C=C1)C(=S)C1=CC=C(S1)C1=CC=C(-*)C=C1,S=C1C2=CC=C(S2)C2=CC=C(C=C2)C2=CC=C1C=C2,1.807,<rdkit.Chem.rdchem.Mol object at 0x7fed97267900>,<rdkit.Chem.rdchem.Mol object at 0x7fed97216820>,<rdkit.Chem.rdchem.Mol object at 0x7fed97222660>
3,4,CO-O-C4H2S-CS,O-C4H2S-CS-CO,*-C(=O)OC1=CC=C(S1)C(-*)=S,*-OC1=CC=C(S1)C(=S)C(-*)=O,O=C1OC2=CC=C(S2)C1=S,1.918,<rdkit.Chem.rdchem.Mol object at 0x7fed972679e0>,<rdkit.Chem.rdchem.Mol object at 0x7fed97216900>,<rdkit.Chem.rdchem.Mol object at 0x7fed97222740>
4,5,C6H4-CS-C4H2S-O,CS-C4H2S-O-C6H4,*-OC1=CC=C(S1)C(=S)C1=CC=C(-*)C=C1,*-C(=S)C1=CC=C(OC2=CC=C(-*)C=C2)S1,S=C1C2=CC=C(OC3=CC=C1C=C3)S2,1.930,<rdkit.Chem.rdchem.Mol object at 0x7fed97267ac0>,<rdkit.Chem.rdchem.Mol object at 0x7fed97216970>,<rdkit.Chem.rdchem.Mol object at 0x7fed972227b0>


In [13]:
# Mol2vec descriptors
def mol2vec(mols, radius=2):
    from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
    
    #Loading pre-trained model via word2vec
    from gensim.models import word2vec
    model = word2vec.Word2Vec.load('./drive/MyDrive/AI/Material Informatics (MI)/Conductive polymer DFT QSAR/model_300dim.pkl')
    sentences =[]
    for idx, mol in enumerate(tqdm(mols)):
      sentences.append(MolSentence(mol2alt_sentence(mol, radius)))
    return pd.DataFrame(sentences2vec(sentences, model, unseen='UNK'))

In [14]:
ru1 = mol2vec(df["mol_ru1"])
ru1 = ru1.dropna(1)

ru2 = mol2vec(df["mol_ru2"])
ru2 = ru2.dropna(1)

rru = mol2vec(df["mol_rru"])
rru = rru.dropna(1)

100%|██████████| 284/284 [00:00<00:00, 1048.75it/s]
<ipython-input-14-2efa449e76fd>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  ru1 = ru1.dropna(1)
100%|██████████| 284/284 [00:00<00:00, 1179.77it/s]
<ipython-input-14-2efa449e76fd>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  ru2 = ru2.dropna(1)
100%|██████████| 284/284 [00:00<00:00, 3418.82it/s]
<ipython-input-14-2efa449e76fd>:8: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  rru = rru.dropna(1)


In [15]:
# A list for the evaluation result
rmse_sum = {}
r2_sum = {}

RU1

In [16]:
from sklearn.model_selection import train_test_split

X = ru1.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [17]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [18]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_ru1: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_ru1: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_ru1: 0.7819513234627009
RMSE(Train): 0.5683669491191764

R_square(Test)_ru1: 0.7704667704952824
RMSE(Test): 0.5035785091650872


In [19]:
rmse_sum['RU1'] = [RMSE_train, RMSE_test] 
r2_sum['RU1'] = [R_square_train, R_square_test] 

RU2

In [20]:
from sklearn.model_selection import train_test_split

X = ru2.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [21]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [22]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_ru2: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_ru2: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_ru2: 0.7611902351286624
RMSE(Train): 0.5948098180937058

R_square(Test)_ru2: 0.7929121657753282
RMSE(Test): 0.4783234768846692


In [23]:
rmse_sum['RU2'] = [RMSE_train, RMSE_test] 
r2_sum['RU2'] = [R_square_train, R_square_test] 

RRU

In [24]:
from sklearn.model_selection import train_test_split

X = rru.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [25]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [26]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_rru: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_rru: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_rru: 0.7877805503757189
RMSE(Train): 0.5607182350036716

R_square(Test)_rru: 0.7974898993511552
RMSE(Test): 0.4730071971914554


In [27]:
rmse_sum['RRU'] = [RMSE_train, RMSE_test] 
r2_sum['RRU'] = [R_square_train, R_square_test] 

Summary

In [28]:
summary = pd.DataFrame(r2_sum, index=["train", "test"])
summary.style.set_caption("Table of R Square (Mol2vec)")

,RU1,RU2,RRU
train,0.781951,0.761190,0.787781
test,0.770467,0.792912,0.797490


In [29]:
summary = pd.DataFrame(rmse_sum, index=["train", "test"])
summary.style.set_caption("Table of RMSE (Mol2vec)")

,RU1,RU2,RRU
train,0.568367,0.594810,0.560718
test,0.503579,0.478323,0.473007
